In [ ]:
pip install dill

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import dill as pickle

from matplotlib.gridspec import GridSpec
from sklearn.calibration import CalibratedClassifierCV, CalibrationDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import roc_curve
from sklearn.metrics import auc

from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools import add_constant

from imblearn.over_sampling import SMOTE
from collections import Counter

In [2]:
data = pd.read_csv("/Users/weiyuna/Desktop/工作/合作项目/医大一-胃癌/模型/code_202203/smote/smote_data.csv")
data_train_smo = data[data.target == "train"]
data_train_smo.reset_index(inplace = True)

In [3]:
def predict_by_model_final(df, is_enable_trend=False):
    data_train_smo = data[data.target == "train"]
    data_train_smo.reset_index(inplace = True)
    
    prior_feature_list = ["gender", "tumor_stage_pN", "cycle"]
    clf_prior = CategoricalNB()
    
    clf_prior_isotonic = CalibratedClassifierCV(clf_prior, cv=2, method="isotonic")
    clf_prior_isotonic_fit = clf_prior_isotonic.fit(data_train_smo[prior_feature_list], data_train_smo["event"])
    
    df["prior"] = clf_prior_isotonic.predict_proba(df[prior_feature_list])[:,1]
    df["prior_odds"] = df["prior"]/(1 - df["prior"])


    quartile_list = ["PLT_q", "Hb_q", "WBC_q", "Neut_q", "Lymph_q"]
    clf_quartile = GaussianNB(priors = [0.5, 0.5]) 
    
    clf_quartile_isotonic = CalibratedClassifierCV(clf_quartile, cv=2, method="isotonic")
    clf_quartile_isotonic_fit = clf_quartile_isotonic.fit(data_train_smo[quartile_list], data_train_smo["event"])
    
    quartile_pre = clf_quartile_isotonic_fit.predict_proba(df[quartile_list])[:,1]
    df["quartile_odds"] = quartile_pre/(1- quartile_pre)
    
    df["posterior_odds"] = df["quartile_odds"]*df["prior_odds"]
    df["posterior"] = df["posterior_odds"]/(1+df["posterior_odds"])  
    return df

In [12]:
import dill as pickle

In [17]:
# save the model
f_save = open('./final_model.pkl', 'wb')
pickle.dump(predict_by_model_final, f_save,protocol=2) ##protocol=2:python2版本
f_save.close()

In [18]:
# reload the model
with open('./final_model.pkl', 'rb') as f:
    predict = pickle.load(f)

In [19]:
data = pd.read_csv("/Users/weiyuna/Desktop/工作/合作项目/医大一-胃癌/模型/code_202203/smote/smote_data.csv")
data_test_smo = data[data.target == "test"]
data_test_smo.reset_index(inplace = True)

# choose a row of data from the test set
data_test_smo_t1 = data_test_smo[1:2]

# output prediction probability
predict(data_test_smo_t1)["posterior"]

/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_13915/3188277971.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prior"] = clf_prior_isotonic.predict_proba(df[prior_feature_list])[:,1]
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_13915/3188277971.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prior_odds"] = df["prior"]/(1 - df["prior"])
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_13915/3188277971.py:22: SettingWithCopyWarning: 
A value is trying to 

1    0.308413
Name: posterior, dtype: float64